In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# %pip install -q sentence-transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
print(os.getcwd())


c:\Users\metet\OneDrive\Documents\GitHub\project-1-2-2023\Project3-1-Group-5\models


In [41]:
train_df = pd.read_csv('datasets/eedi-mining-misconceptions-in-mathematics/train.csv')
train_df

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864,1864,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,C,What is the range of the following numbers?\n\...,\( 5 \),\( 11 \),\( 23 \),\( 16 \),2456.0,691.0,NaN,1349.0
1865,1865,2695,"Describe an enlargement, with no centre of enl...",90,Length Scale Factors in Similar Shapes,B,Shape \( Q \) is an enlargement of shape \( P ...,\( 3 \div 11 \),\( 11 \div 3 \),\( 3 \times 11 \),\( 11-3 \),1500.0,NaN,2442.0,1258.0
1866,1866,854,Use the order of operations to carry out calcu...,33,BIDMAS,B,What does the following equal?\n\[\n8-7+10 \ti...,\( 36 \),\( 31 \),\( -29 \),\( 33 \),NaN,NaN,2306.0,1507.0
1867,1867,2634,Distinguish between congruency and similarity,274,Congruency in Other Shapes,B,Tom and Katie are discussing congruence and si...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,2312.0,NaN,2312.0,2312.0


In [42]:
questions = train_df['QuestionText'].tolist()  # All question texts
misconceptions = []
correct_answers = []

In [43]:
# Loop through each row in the training DataFrame to build the 2D misconceptions array
for _, row in train_df.iterrows():
    question_misconceptions = []
    # Track the correct answer for this question
    correct_answer = row['CorrectAnswer']
    correct_answers.append(correct_answer)  # Store correct answer separately

    # Iterate over answer choices and add misconception IDs
    for answer, misconception_id in zip(['AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText'],
                                        ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']):
        misconception_value = row[misconception_id]
        # Add the misconception ID if it is not NaN; else, add None
        question_misconceptions.append(misconception_value if pd.notna(misconception_value) else None)
    
    # Append the current question's misconceptions (A, B, C, D) as one row in the misconceptions array
    misconceptions.append(question_misconceptions)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Load the SBERT model
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can use any SBERT model

# Prepare SBERT embeddings
def get_sbert_embeddings(texts):
    return sbert_model.encode(texts, convert_to_numpy=True)

ModuleNotFoundError: No module named 'sentence_transformers'

In [22]:
# Prepare data and labels for training
data = []
labels = []

# Populate data with each question-answer pair, labeling each with the corresponding misconception ID
for question_text, misconceptions_row, correct_answer in zip(questions, misconceptions, correct_answers):
    for answer_index, misconception_id in enumerate(misconceptions_row):
        # Skip if this is the correct answer or misconception_id is NaN
        if misconception_id is None or answer_index == ord(correct_answer) - ord('A'):
            continue

        # Concatenate question text with answer choice text
        answer_text = train_df.iloc[0][f'Answer{chr(ord("A") + answer_index)}Text']
        full_text = f"{question_text} {answer_text}"
        data.append(full_text)
        labels.append(misconception_id)


In [23]:
# Encode labels to integers for compatibility with classifiers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Generate SBERT embeddings for the training data
data_embeddings = get_sbert_embeddings(data)

# Train a logistic regression model on the SBERT embeddings
model = LogisticRegression(max_iter=1000)
model.fit(data_embeddings, labels_encoded)

Batches:   0%|          | 0/137 [00:00<?, ?it/s]

LogisticRegression(max_iter=1000)

In [45]:
# Prepare test data (modify this based on the structure of your test data)
test_data = []  # Store test question-answer pairs
test_question_ids = []  # Store corresponding QuestionId and answer index
test_df = pd.read_csv('datasets/eedi-mining-misconceptions-in-mathematics/test.csv')  # Load your test data here

In [ ]:
def map_at_25(y_true, y_pred):
    """
    Calculate Mean Average Precision at 25 
    Args:
        y_true: List of true misconceptions for each sample (list of lists).
        y_pred: List of predicted misconceptions for each sample (list of lists).
    Returns:
        MAP@25 score.
    """
    map_score = 0
    for true_labels, predictions in zip(y_true, y_pred):
        predictions = predictions[:25]  
        score = 0.0
        hits = 0
        for i, pred in enumerate(predictions, start=1):
            if pred in true_labels:
                hits += 1
                score += hits / i  # Precision at i
        if true_labels:
            map_score += score / min(len(true_labels), 25)  # Normalize by number of true labels
    return map_score / len(y_true)
def mean_reciprocal_rank(y_true, y_pred):
    """
    Calculate Mean Reciprocal Rank (MRR).
    Args:
        y_true: List of true misconceptions for each sample (list of lists).
        y_pred: List of predicted misconceptions for each sample (list of lists).
    Returns:
        MRR score.
    """
    reciprocal_ranks = []
    for true_labels, predictions in zip(y_true, y_pred):
        for i, pred in enumerate(predictions, start=1):
            if pred in true_labels:
                reciprocal_ranks.append(1 / i)
                break
        else:
            reciprocal_ranks.append(0)  # If no correct prediction, reciprocal rank is 0
    return np.mean(reciprocal_ranks)

def precision_at_k(y_true, y_pred, k=25):
    """
    Calculate Precision@K.
    Args:
        y_true: List of true misconceptions for each sample (list of lists).
        y_pred: List of predicted misconceptions for each sample (list of lists).
        k: Number of top predictions to consider.
    Returns:
        Precision@K score.
    """
    precisions = []
    for true_labels, predictions in zip(y_true, y_pred):
        predictions = predictions[:k]  # Only consider top K predictions
        hits = sum(1 for pred in predictions if pred in true_labels)
        precisions.append(hits / k)
    return np.mean(precisions)



In [ ]:
for _, row in test_df.iterrows():
    question_text = row['QuestionText']
    correct_answer = row['CorrectAnswer']

    for answer_index in range(4):
        # Skip the correct answer
        if answer_index == ord(correct_answer) - ord('A'):
            continue

        # Form the text for prediction
        answer_text = row[f'Answer{chr(ord("A") + answer_index)}Text']
        full_text = f"{question_text} {answer_text}"
        test_data.append(full_text)

        # Store identifier for the output format
        test_question_ids.append(f"{row['QuestionId']}_{chr(ord('A') + answer_index)}")

# Generate SBERT embeddings for the test data
test_data_embeddings = get_sbert_embeddings(test_data)

# Predict probabilities for each question-answer pair in the test data
predicted_probabilities = model.predict_proba(test_data_embeddings)

# Get the class (misconception ID) labels for reference
misconception_classes = label_encoder.inverse_transform(model.classes_)

output = []

for question_id, probabilities in zip(test_question_ids, predicted_probabilities):
    # Sort the misconceptions by descending probability and select the top 25
    top_25_indices = np.argsort(probabilities)[-25:][::-1]
    top_25_misconceptions = misconception_classes[top_25_indices].tolist()

    # Append the question ID and top 25 misconceptions as a list
    output.append([question_id, top_25_misconceptions])

y_true = train_df[['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']].values.tolist()
y_true = [[int(y) for y in row if pd.notna(y)] for row in y_true]  

y_pred = []
for question_id, probabilities in zip(test_question_ids, predicted_probabilities):
    top_25_indices = np.argsort(probabilities)[-25:][::-1]
    y_pred.append(misconception_classes[top_25_indices].tolist())

map25_score = map_at_25(y_true, y_pred)
print(f"MAP@25: {map25_score}")

mrr_score = mean_reciprocal_rank(y_true, y_pred)
print(f"MRR: {mrr_score}")

precision_25_score = precision_at_k(y_true, y_pred, k=25)
print(f"Precision@25: {precision_25_score}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
output_df = pd.DataFrame(output, columns=['QuestionId_Answer', 'MisconceptionId'])
# Convert each list in the 'MisconceptionIds' column to a space-separated string# Convert each list in 'MisconceptionIds' to a space-separated string of integers
output_df['MisconceptionId'] = output_df['MisconceptionId'].apply(lambda ids: ' '.join(map(lambda x: str(int(x)), ids)))

# Save the modified DataFrame to a CSV file
output_df.to_csv('final_predicted_misconceptions.csv', index=False, header=['QuestionId_Answer', 'MisconceptionId'])

# Save the modified DataFrame to a CSV file
output_df.to_csv('submission.csv', index=False, header=['QuestionId_Answer', 'MisconceptionId'])


In [27]:
output_df

,QuestionId_Answer,MisconceptionId
0,1869_B,1507 2306 1672 1054 706 234 1510 2532 974 1214...
1,1869_C,1507 2306 1054 1672 706 234 1510 2532 974 1214...
2,1869_D,1672 1507 706 2306 1054 2532 1510 234 1383 974...
3,1870_A,143 2142 1072 891 2078 1755 2398 1644 167 363 ...
4,1870_B,143 2142 1072 891 2078 2398 1755 167 363 1644 ...
5,1870_C,143 2142 1072 891 1755 2078 167 363 2398 113 1...
6,1871_A,1287 1073 1072 1978 1530 686 2561 1214 602 363...
7,1871_C,1287 1073 1072 1978 2561 1530 686 602 1214 186...
8,1871_D,1287 1073 1978 1072 1214 1530 2561 686 1471 13...
